you can download the data here  https://drive.google.com/drive/folders/1mZC3NuPBZC4mjP3_kH18c9fLrv8ME7RU

In [ ]:
# Create Delta Table
import duckdb 
from deltalake.writer import write_deltalake
con = duckdb.connect()
result = con.execute( '''
         select *, extract ( year from l_shipdate) as year from 'C:/Users/mimoune.djouallah/Desktop/TPC-H-SF10/parquet/lineitem.parquet' 
         ''').arrow()

write_deltalake("C:/Users/mimoune.djouallah/Desktop/TPC-H-SF10/parquet/lineitem", result,mode='append',partition_by=['year'])

In [10]:
#Query delta Table
import duckdb 
from deltalake import DeltaTable

lineitem = DeltaTable("/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/lineitem").to_pyarrow_dataset()

con = duckdb.connect()
df =con.execute('''
SELECT
    --Query01
    l_returnflag,
    l_linestatus,
    SUM(l_quantity) AS sum_qty,
    SUM(l_extendedprice) AS sum_base_price,
    SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    AVG(l_quantity) AS avg_qty,
    AVG(l_extendedprice) AS avg_price,
    AVG(l_discount) AS avg_disc,
    COUNT(*) AS count_order
FROM
    lineitem
WHERE
    cast(l_shipdate as date) = '1998-09-02'
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
''').df()
df

,L_RETURNFLAG,L_LINESTATUS,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,N,O,479297.0,7.219848e+08,6.858627e+08,7.134579e+08,25.561143,38503.800129,0.049926,18751


In [ ]:
#CREATE HIVE table
import duckdb 
from deltalake.writer import write_deltalake
import pyarrow as pa
con = duckdb.connect()
result = con.execute( '''
         select *, extract ( year from l_shipdate) as year from 'C:/Users/mimoune.djouallah/Desktop/TPC-H-SF10/parquet/lineitem.parquet'
         ''').arrow()

pa.dataset.write_dataset(result,base_dir="/Users/mimoune.djouallah/Desktop/TPC-H-SF10/parquet/lineitem2", format ='parquet',partitioning=['year'],partitioning_flavor="hive")

In [8]:
#QUERY using native hive support in DuckDB
import duckdb 

con = duckdb.connect()


df =con.execute('''

PRAGMA enable_profiling ;

SELECT
    --Query01
    l_returnflag,
    l_linestatus,
    SUM(l_quantity) AS sum_qty,
    SUM(l_extendedprice) AS sum_base_price,
    SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    AVG(l_quantity) AS avg_qty,
    AVG(l_extendedprice) AS avg_price,
    AVG(l_discount) AS avg_disc,
    COUNT(*) AS count_order
FROM   read_parquet('/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/lineitem2/*/*.parquet', HIVE_PARTITIONING=1)
WHERE
    l_shipdate = '1998-09-02' 
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
''').df()
df

,L_RETURNFLAG,L_LINESTATUS,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,N,O,479297.0,7.219848e+08,6.858627e+08,7.134579e+08,25.561143,38503.800129,0.049926,18751


In [2]:
#Query using  Pyarrow Dataset
import duckdb 
import pyarrow.dataset as ds
con = duckdb.connect()
lineitem = ds.dataset("/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/lineitem2/",format="parquet", partitioning="hive")


df =con.execute('''

PRAGMA enable_profiling ;

SELECT
    --Query01
    l_returnflag,
    l_linestatus,
    SUM(l_quantity) AS sum_qty,
    SUM(l_extendedprice) AS sum_base_price,
    SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    AVG(l_quantity) AS avg_qty,
    AVG(l_extendedprice) AS avg_price,
    AVG(l_discount) AS avg_disc,
    COUNT(*) AS count_order
FROM
    lineitem
WHERE
    l_shipdate = '1998-09-02' 
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
''').df()
df



,L_RETURNFLAG,L_LINESTATUS,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,N,O,479297.0,7.219848e+08,6.858627e+08,7.134579e+08,25.561143,38503.800129,0.049926,18751


In [11]:
#Query using single parquet file
import duckdb 

con = duckdb.connect()


df =con.execute('''

PRAGMA enable_profiling ;

SELECT
    --Query01
    l_returnflag,
    l_linestatus,
    SUM(l_quantity) AS sum_qty,
    SUM(l_extendedprice) AS sum_base_price,
    SUM(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    AVG(l_quantity) AS avg_qty,
    AVG(l_extendedprice) AS avg_price,
    AVG(l_discount) AS avg_disc,
    COUNT(*) AS count_order
FROM    read_parquet('/Users/mimoune.djouallah/Desktop/TPC-H-SF10/Parquet/lineitem.parquet')
WHERE
    l_shipdate = '1998-09-02' 
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
''').df()
df

,L_RETURNFLAG,L_LINESTATUS,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,N,O,479297.0,7.219848e+08,6.858627e+08,7.134579e+08,25.561143,38503.800129,0.049926,18751


if you have small data, maybe a parquet file with less than 2 GB, probabaly partition will not help you much